In [15]:
from pycalphad import Database, Model
from symengine import Lambdify, sympify
from pycalphad.codegen.sympydiff_utils import build_functions
dbf = Database('AlBeCuZrO.tdb')
mod = Model(dbf, ['CU', 'ZR', 'VA'], 'IONIC_LIQ')

In [17]:
sympify(mod.variables)

[T, IONIC_LIQ0CU_POS1, IONIC_LIQ0CU_POS2, IONIC_LIQ0ZR_POS4, IONIC_LIQ1VA]

In [10]:
%%timeit
func = Lambdify(mod.variables, [mod.GM], backend='llvm', real=True, cse=True, order='C')

73.6 ms ± 321 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
import numpy as np
inp = np.array([[300, 0.25, 0.25, 0.25, 0.25]])
n = int(5e5)
inp = np.repeat(inp, n).reshape((-1, 5), order='C')

out = np.zeros((n,), order='C')

In [12]:
mod.variables

[T, IONIC_LIQ0CU_POS1, IONIC_LIQ0CU_POS2, IONIC_LIQ0ZR_POS4, IONIC_LIQ1VA]

In [13]:
# Suboptimal because this can be done in a Cython loop
func = Lambdify(mod.variables, [mod.GM], backend='llvm', real=True, cse=True, order='C')
flattened_inp = inp.ravel()
for idx in range(inp.shape[0]):
    func.unsafe_real(flattened_inp, out, idx*inp.shape[1], idx)

In [14]:
%timeit out = func(inp)
out = func(inp)

104 ms ± 228 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit
func_orig = build_functions(mod.GM, mod.variables, wrt=None,
                            include_obj=True, include_grad=False, include_hess=False,
                            parameters=None)
func_orig.func.compile()

1.19 s ± 34.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
func_orig = build_functions(mod.GM, mod.variables, wrt=None,
                            include_obj=True, include_grad=False, include_hess=False,
                            parameters=None)
func_orig.func.compile()
%timeit out_orig = func_orig.func.kernel(inp, np.array([0.]))

348 ms ± 4.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
out_orig = func_orig.func.kernel(inp, np.array([0.]))
np.testing.assert_allclose(out[:,0], out_orig)